# RNPDNO publico

In [1]:
import json
import flatdict
import pandas as pd
import warnings

In [2]:
def formatear_registro(registros):
    reportes_ordenados = dict()
    list_reportes = list()
    consecutivo = registros['Consecutivo']
    for i_repo, repo in enumerate(registros['reportes']):
        data_repo = {'Consecutivo': consecutivo, 'Numero reporte':i_repo}
        for k_campo, v_campo in repo.items():
            if k_campo=='Tipo Reporte':
                data_repo['Tipo Reporte'] = v_campo
            """
            s_involucradas = 'DATOS DE LA(S) PERSONA(S) PROBABLEMENTE INVOLUCRADA(S)'
            if k_campo==s_involucradas:
                v_campo_per = dict()
                for i, persona in enumerate(v_campo):
                    for k,v in persona["Persona"].items():
                        v_campo_per[f"{k}_Persona {i}"] = v
                v_campo = v_campo_per
            """
            if type(v_campo)==list:
                v_campo = {i:v for i,v in enumerate(v_campo)}
            if type(v_campo)==dict: # campos con diccionarios
                data_repo[k_campo] = dict()
                # iterar sobre subcampos
                for k_sub, v_sub in v_campo.items():
                    if type(v_sub)==list and len(v_sub)==1:
                        v_sub = v_sub[0]
                    subcampos = list()
                    try:
                        for inst, data_sub in v_sub.items():
                            data_sub = flatten_subcampo(data_sub)
                            data_sub['Institucion'] = inst
                            subcampos.append(data_sub)
                    except:
                        print('!', k_campo, k_sub, v_sub)
                    data_repo[k_campo][k_sub] = subcampos
        list_reportes.append(data_repo)
    return list_reportes

def dict_everything(data_structure):
    if type(data_structure) == list:
        data_structure = {str(i):v for i,v in enumerate(data_structure)}
    if type(data_structure) == dict:
        data_structure = {k:dict_everything(v) for k,v in data_structure.items()}
    return data_structure

def flatten_subcampo(data, reverse_flatten_keys=True):
    def reverse(s, delimiter='_'):
        if type(s)==str and '_' in s:
            s = '_'.join( s.split('_')[::-1])
        return s
    data = dict_everything(data)
    data = flatdict.FlatDict(data, delimiter='_')
    if reverse_flatten_keys:
        data = {reverse(k):v for k,v in data.items()}
    return data

file = 'data_raw/Json01_Id01.json'
with open(file) as f:
    data_all = json.load(f)
len(data_all)

registro = data_all[40]
registro = formatear_registro(registro)
registro

[{'Consecutivo': 41,
  'Numero reporte': 0,
  'Tipo Reporte': 'DATOS PÚBLICOS PREVIO A LA PUESTA EN OPERACIÓN DEL RNPDNO',
  'DATOS DE LA PERSONA DESAPARECIDA O\xa0NO\xa0LOCALIZADA': {'DATOS DEL REPORTE': [{'Medio de Conocimiento_1': 'REPORTE',
     'A través de_1': 'SIN DATO',
     '¿La Persona que reporta, desea que sus datos sean anónimos?_1': 'NO',
     'Estatus  de la víctima_1': 'DESAPARECIDA',
     'Expediente_1': 'SIN DATO',
     'Fecha de inicio del expediente_1': 'SIN DATO',
     'Hipótesis de la No Localización_1': 'SIN DATO',
     'Delito(s)_1': 'SIN DATO',
     'Institucion': 'FISCALIA\xa0GENERAL DE SONORA'}],
   'DATOS GENERALES': [{'Nombre _1': 'JOSE DOLORES',
     'Primer apellido _1': 'GUTIERREZ',
     'Segundo apellido_1': 'RASCON',
     'Sexo _1': 'HOMBRE',
     'Género_1': 'MASCULINO',
     'Nacionalidad _1': 'MEXICANA',
     'CURP_1': 'ELIMINADO-15',
     'RFC._1': 'SIN DATO',
     'Estado civil_1': 'SIN DATO',
     'Lugar de nacimiento_1': 'SIN DATO',
     'Fecha 

In [ ]:
def contar_valores(data_sub, return_list=False):
    res = {'Total':0, 'ConDato':0, 'Eliminado':0, 'SinDato':0,}
    res['Total'] = len(data_sub)
    for k,v in data_sub.items():
            if v=='SIN DATO': 
                res['SinDato'] += 1
            elif type(v)==str and v.startswith('ELIMINADO'): 
                res['Eliminado'] += 1
            else:
                res['ConDato'] += 1
    if return_list:
            res = res.values()
    return res

file = 'data_raw/Json01_Id01.json'
with open(file) as f:
    data_all = json.load(f)
len(data_all)
json_id = '01'

summary = list()

data_formateada = dict()
for registro in data_all:
        registro = formatear_registro(registro) #regresa lista de dics
        
        for repo in registro: #itera la lista
            repo_id = str(repo['Consecutivo']).zfill(6)
            id_reporte = f"{json_id}-{repo_id}-{repo['Numero reporte']}"
            print(f"Reporte {id_reporte}")
            data_formateada[id_reporte] = repo
            
            # itera los campos del reporte
            for k_campo, v_campo in repo.items():
                if type(v_campo)==dict:
                    
                    # itera los subcampos
                    for k_sub, v_sub in v_campo.items():
                        # navegar adentro de la lista de diccionarios de los subcampo
                        for i, data_sub in enumerate(v_sub):
                            res_data = {
                                        'id_reporte': id_reporte,
                                        'campo':k_campo,
                                        'subcampo':str(k_sub),
                                        'n_subcampo':i,
                                        'institucion':data_sub['Institucion']
                                        }
                            res_count = contar_valores(data_sub) # contar valores
                            summary.append( {**res_data, **res_count} )


df = pd.DataFrame(summary)
df

In [4]:

import pandas as pd
from ydata_profiling import ProfileReport

file_profile = "RNPDNO_json1_minprofile.html"
prof = df.profile_report(
                         title=file_profile, #title
                         minimal=True, #Profile minimo para conjuntos de datos grandes
                         correlations=None, #Quitar correlaciones
                        )
prof.to_file(output_file=file_profile)

/home/esther/anaconda3/envs/RNPDNO_publico/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|█████████████████████| 1/1 [00:00<00:00, 540.92it/s]


In [5]:
# File path to save JSON
f_out = "Json01_Id01_flat.json"

# Writing dictionary to JSON file
with open(f_out, "w") as json_file:
    json.dump(data_formateada, json_file)
    
# File path to save CSV
f_out = "Json01_Id01_count.csv"

df.to_csv(f_out, index=False)

In [11]:
df[['campo','subcampo']].value_counts()

campo                                                   subcampo                                                   
DATOS DE LA PERSONA DESAPARECIDA O NO LOCALIZADA        DATOS DEL REPORTE                                              6138
                                                        DATOS GENERALES                                                5896
                                                        HECHOS                                                         5825
                                                        LUGAR DE LOS HECHOS                                            5749
                                                        MEDIA FILIACION                                                5740
DATOS DE LA AUTORIDAD                                   DATOS GENERALES                                                5738
DATOS DE LA PERSONA DESAPARECIDA O NO LOCALIZADA        FOTOGRAFIAS                                                    5727
                